# TFM | Reinforcement Learning | Daniel Zorrilla

## 1. Import Dependencies
This is an example

In [1]:
!pip install stable-baselines3[extra]
!pip install pyglet

**OS** - Operative System
**Gym** - Open AI Gym
**stable_baselines3** - allows us to vectorize environments. Train machine learning model. Train RL Agent

In [2]:
import os
import gym
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy #Test how well a model is performing

## 2. Environments

In [34]:
environment_name = 'CartPole-v0'
env = gym.make(environment_name)

In [35]:
episodes = 5
for episode in range (1, episodes+1):
    state = env.reset()
    done = False
    score = 0
    
    while not done:
        env.render()
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score += reward
    print('Episode:{} Score:{}'.format(episode,score))

Episode:1 Score:16.0
Episode:2 Score:40.0
Episode:3 Score:45.0
Episode:4 Score:20.0
Episode:5 Score:12.0


In [36]:
env.close()

In [37]:
env.action_space

Discrete(2)

In [38]:
env.action_space.sample()

1

In [39]:
env.observation_space

Box([-4.8000002e+00 -3.4028235e+38 -4.1887903e-01 -3.4028235e+38], [4.8000002e+00 3.4028235e+38 4.1887903e-01 3.4028235e+38], (4,), float32)

In [40]:
env.observation_space.sample()

array([-1.8568482e-01, -3.1553590e+38, -1.3480744e-01,  2.4828505e+38],
      dtype=float32)

## 3. Train and RL Model

In [41]:
log_path = os.path.join('Training', 'Logs') #where we save our tensorboard log

In [42]:
log_path

'Training\\Logs'

In [43]:
!pip3 install torch torchvision torchaudio

In [44]:
env = gym.make(environment_name)
env = DummyVecEnv([lambda: env]) #Wrapped environment using DummyVecEnv
model = PPO('MlpPolicy', env, verbose = 1, tensorboard_log=log_path)

Using cpu device


In [45]:
#PPO?? #see all the parameter that can be trained for the algorithm

In [48]:
model.learn(total_timesteps=20000) #selfdriving 200000 or 300000 steps Training the model model.learn

Logging to Training\Logs\PPO_3
-----------------------------
| time/              |      |
|    fps             | 1586 |
|    iterations      | 1    |
|    time_elapsed    | 1    |
|    total_timesteps | 2048 |
-----------------------------
------------------------------------------
| time/                   |              |
|    fps                  | 1057         |
|    iterations           | 2            |
|    time_elapsed         | 3            |
|    total_timesteps      | 4096         |
| train/                  |              |
|    approx_kl            | 0.0047873463 |
|    clip_fraction        | 0.0339       |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.502       |
|    explained_variance   | 0.417        |
|    learning_rate        | 0.0003       |
|    loss                 | 0.0187       |
|    n_updates            | 210          |
|    policy_gradient_loss | -0.00255     |
|    value_loss           | 0.118        |
----------------------------

## 4. Save and Reload Model

In [49]:
PPO_Path = os.path.join('Training', 'Saved Models', 'PPO_Model_Cartpole')

In [50]:
model.save(PPO_Path) #save model

In [51]:
del model #Delete model to simulate reloading in production

In [52]:
model = PPO.load(PPO_Path, env = env)

## 5. Evaluation

In [53]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)

C:\Users\Daniel\anaconda3\lib\site-packages\stable_baselines3\common\evaluation.py:65: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


(200.0, 0.0)

In [54]:
env.close()

## 6. Test Model

In [55]:
episodes = 5
for episode in range (1, episodes+1):
    obs = env.reset()
    done = False
    score = 0
    
    while not done:
        env.render()
        action, _ = model.predict(obs) #using trainned model to predict actions
        obs, reward, done, info = env.step(action)
        score += reward
    print('Episode:{} Score:{}'.format(episode,score))

Episode:1 Score:[200.]
Episode:2 Score:[200.]
Episode:3 Score:[200.]
Episode:4 Score:[200.]
Episode:5 Score:[200.]


In [56]:
env.close()

## 7. Viewing Logs in TensorBoard

In [57]:
training_log_path = os.path.join(log_path, 'PPO_3')

In [58]:
training_log_path

'Training\\Logs\\PPO_3'

In [59]:
!tensorboard --logdir={training_log_path} #Execute in command line the tensorboard visualization http://localhost:6006
                                          #stop the cell to continue

^C


## 8. Adding a callback to the training stage

In [60]:
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold

In [61]:
save_path = os.path.join('Training','Saved Models') #Where the best model is going to be saved

In [62]:
stop_callback = StopTrainingOnRewardThreshold(reward_threshold=200, verbose=1) #Stop our training when we achieved a 200 rwd
eval_callback = EvalCallback(env,  #callback that is triggered after each training run
                            callback_on_new_best=stop_callback, #callback to run in the new best model
                            eval_freq=10000, #Evaluation Frequency to 10.000 time steps
                            best_model_save_path=save_path, # Save the model everytime there is a new best model
                            verbose=1)

In [63]:
model = PPO('MlpPolicy', env, verbose=1, tensorboard_log=log_path)

Using cpu device


In [64]:
model.learn(total_timesteps=20000, callback=eval_callback)

Logging to Training\Logs\PPO_4
-----------------------------
| time/              |      |
|    fps             | 1548 |
|    iterations      | 1    |
|    time_elapsed    | 1    |
|    total_timesteps | 2048 |
-----------------------------
----------------------------------------
| time/                   |            |
|    fps                  | 923        |
|    iterations           | 2          |
|    time_elapsed         | 4          |
|    total_timesteps      | 4096       |
| train/                  |            |
|    approx_kl            | 0.00872235 |
|    clip_fraction        | 0.0979     |
|    clip_range           | 0.2        |
|    entropy_loss         | -0.686     |
|    explained_variance   | 0.00251    |
|    learning_rate        | 0.0003     |
|    loss                 | 6.81       |
|    n_updates            | 10         |
|    policy_gradient_loss | -0.0134    |
|    value_loss           | 51.1       |
----------------------------------------
---------------------

C:\Users\Daniel\anaconda3\lib\site-packages\stable_baselines3\common\evaluation.py:65: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


Eval num_timesteps=10000, episode_reward=149.80 +/- 51.79
Episode length: 149.80 +/- 51.79
-----------------------------------------
| eval/                   |             |
|    mean_ep_length       | 150         |
|    mean_reward          | 150         |
| time/                   |             |
|    total_timesteps      | 10000       |
| train/                  |             |
|    approx_kl            | 0.009578235 |
|    clip_fraction        | 0.0681      |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.612      |
|    explained_variance   | 0.314       |
|    learning_rate        | 0.0003      |
|    loss                 | 17.5        |
|    n_updates            | 40          |
|    policy_gradient_loss | -0.0145     |
|    value_loss           | 59.5        |
-----------------------------------------
New best mean reward!
------------------------------
| time/              |       |
|    fps             | 656   |
|    iterations      | 5     |
|    ti

## 9. Changing Policies

In [70]:
net_arch = [dict(pi=[128,128,128,128], vf=[128,128,128,128])] #dictionary neural network for our custom actor=PI and valueFunctn
                                                              #128 un/eachLayer (4Lyrs)

In [71]:
#associating this new_Arch to the model
model = PPO('MlpPolicy', env, verbose=1, tensorboard_log=log_path, policy_kwargs={'net_arch':net_arch})

Using cpu device


In [72]:
model.learn(total_timesteps=20000, callback=eval_callback)

Logging to Training\Logs\PPO_5
-----------------------------
| time/              |      |
|    fps             | 1043 |
|    iterations      | 1    |
|    time_elapsed    | 1    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 640         |
|    iterations           | 2           |
|    time_elapsed         | 6           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.015336603 |
|    clip_fraction        | 0.22        |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.681      |
|    explained_variance   | -0.00715    |
|    learning_rate        | 0.0003      |
|    loss                 | 4.63        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0274     |
|    value_loss           | 23.8        |
-----------------------------------------
---

C:\Users\Daniel\anaconda3\lib\site-packages\stable_baselines3\common\evaluation.py:65: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


Eval num_timesteps=10000, episode_reward=200.00 +/- 0.00
Episode length: 200.00 +/- 0.00
-----------------------------------------
| eval/                   |             |
|    mean_ep_length       | 200         |
|    mean_reward          | 200         |
| time/                   |             |
|    total_timesteps      | 10000       |
| train/                  |             |
|    approx_kl            | 0.011491954 |
|    clip_fraction        | 0.121       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.565      |
|    explained_variance   | 0.483       |
|    learning_rate        | 0.0003      |
|    loss                 | 14.7        |
|    n_updates            | 40          |
|    policy_gradient_loss | -0.0181     |
|    value_loss           | 43.8        |
-----------------------------------------
------------------------------
| time/              |       |
|    fps             | 410   |
|    iterations      | 5     |
|    time_elapsed    | 24    |


## 10. Using DQN Algorithm

In [73]:
from stable_baselines3 import DQN

In [74]:
model = DQN('MlpPolicy', env, verbose=1, tensorboard_log=log_path)

Using cpu device


In [75]:
model.learn(total_timesteps=20000)

Logging to Training\Logs\DQN_1
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.97     |
| time/               |          |
|    episodes         | 4        |
|    fps              | 2664     |
|    time_elapsed     | 0        |
|    total_timesteps  | 64       |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.942    |
| time/               |          |
|    episodes         | 8        |
|    fps              | 3386     |
|    time_elapsed     | 0        |
|    total_timesteps  | 122      |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.899    |
| time/               |          |
|    episodes         | 12       |
|    fps              | 4173     |
|    time_elapsed     | 0        |
|    total_timesteps  | 213      |
----------------------------------
------------------------

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 108      |
|    fps              | 4722     |
|    time_elapsed     | 0        |
|    total_timesteps  | 2382     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 112      |
|    fps              | 4714     |
|    time_elapsed     | 0        |
|    total_timesteps  | 2458     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 116      |
|    fps              | 4760     |
|    time_elapsed     | 0        |
|    total_timesteps  | 2563     |
----------------------------------
----------------------------------
| rollout/          

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 216      |
|    fps              | 5199     |
|    time_elapsed     | 0        |
|    total_timesteps  | 4813     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 220      |
|    fps              | 5237     |
|    time_elapsed     | 0        |
|    total_timesteps  | 4958     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 224      |
|    fps              | 5267     |
|    time_elapsed     | 0        |
|    total_timesteps  | 5081     |
----------------------------------
----------------------------------
| rollout/          

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 324      |
|    fps              | 5188     |
|    time_elapsed     | 1        |
|    total_timesteps  | 7145     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 328      |
|    fps              | 5167     |
|    time_elapsed     | 1        |
|    total_timesteps  | 7204     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 332      |
|    fps              | 5159     |
|    time_elapsed     | 1        |
|    total_timesteps  | 7270     |
----------------------------------
----------------------------------
| rollout/          

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 432      |
|    fps              | 5203     |
|    time_elapsed     | 1        |
|    total_timesteps  | 9436     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 436      |
|    fps              | 5190     |
|    time_elapsed     | 1        |
|    total_timesteps  | 9505     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 440      |
|    fps              | 5171     |
|    time_elapsed     | 1        |
|    total_timesteps  | 9564     |
----------------------------------
----------------------------------
| rollout/          

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 540      |
|    fps              | 5195     |
|    time_elapsed     | 2        |
|    total_timesteps  | 11984    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 544      |
|    fps              | 5190     |
|    time_elapsed     | 2        |
|    total_timesteps  | 12066    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 548      |
|    fps              | 5182     |
|    time_elapsed     | 2        |
|    total_timesteps  | 12136    |
----------------------------------
----------------------------------
| rollout/          

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 648      |
|    fps              | 5183     |
|    time_elapsed     | 2        |
|    total_timesteps  | 14172    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 652      |
|    fps              | 5187     |
|    time_elapsed     | 2        |
|    total_timesteps  | 14260    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 656      |
|    fps              | 5187     |
|    time_elapsed     | 2        |
|    total_timesteps  | 14312    |
----------------------------------
----------------------------------
| rollout/          

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 756      |
|    fps              | 5264     |
|    time_elapsed     | 3        |
|    total_timesteps  | 16610    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 760      |
|    fps              | 5265     |
|    time_elapsed     | 3        |
|    total_timesteps  | 16688    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 764      |
|    fps              | 5267     |
|    time_elapsed     | 3        |
|    total_timesteps  | 16773    |
----------------------------------
----------------------------------
| rollout/          

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 864      |
|    fps              | 5337     |
|    time_elapsed     | 3        |
|    total_timesteps  | 19239    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 868      |
|    fps              | 5339     |
|    time_elapsed     | 3        |
|    total_timesteps  | 19322    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 872      |
|    fps              | 5344     |
|    time_elapsed     | 3        |
|    total_timesteps  | 19410    |
----------------------------------
----------------------------------
| rollout/          

In [76]:
DQN_Path = os.path.join('Training','Saved Models', 'DQN_Model_Cartpole')
model.save(DQN_Path)